In [40]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
#from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression

In [41]:
data = pd.read_csv("BANKNIFTYFUTALL.csv")
feature_cols_JUL = ['trade', 'event', 'ob', 'bevent','btrade','spreadreturn', 'niftyidevent','niftyidindex']
#data.dropna(subset=['return800'], axis=0)

data.drop([data.index[457897]],inplace=True)
#data.isnull().sum(axis = 0)

##### Create n samples from the dataset and perform regression on each sample and take the average of the score of each samples

In [42]:
n=10
sco=[]
while(n):
    col=random.randint(2,7)
    row=random.randint(228948,457897)
    col_list=[]
    for i in range(len(feature_cols_JUL)):
        if(i<=col):
            col_list.append(feature_cols_JUL[i])
    XX=data[col_list]
    X=XX.iloc[0:row,:]
    Y=data.iloc[0:row,34]
    reg=LinearRegression()
    reg.fit(X,Y)
    r_sq = reg.score(X,Y)
    sco.append(r_sq)
    n=n-1
print(sco)
s=0
for i in sco:
    s+=i
print(s/(len(sco)))

[0.004289760088394212, 0.0029516989321035636, 0.003705461271462429, 0.0034769729626544787, 0.0029830959093758436, 0.003769709727808324, 0.0036027079108490456, 0.0026740858531143497, 0.004146315573728798, 0.004178450341793205]
0.003577825857128425


##### Calculating average weights by taking sub-samples from the dataset and then predicting accordingly

In [4]:
n=10
sco=[]
dcount={}
davg={}
for i in feature_cols_JUL:
    dcount.update({i : 0})
    davg.update({i : 0})
while(n):
    col=random.randint(1,7)
    row=random.randint(0,457897)
    col_list=[]
    if(n%2==0):
        feature_cols_JUL.reverse()
    for i in range(len(feature_cols_JUL)):
        if(i<=col):
            col_list.append(feature_cols_JUL[i])
    XX=data[col_list]
    X=XX.iloc[0:row,:]
    Y=data.iloc[0:row,34]
    reg=LinearRegression()
    reg.fit(X,Y)
    Coef = pd.Series(reg.coef_,index=col_list)
    #print(Coef)
    #print()
    for index, value in Coef.items():
        dcount[index]+=1
        davg[index]+=value
    n=n-1
print(dcount)
print(davg)
for key,val in davg.items():
    temp= dcount[key]
    davg[key]=val/temp
print(davg)

{'trade': 4, 'event': 6, 'ob': 5, 'bevent': 6, 'btrade': 6, 'spreadreturn': 7, 'niftyidevent': 8, 'niftyidindex': 7}
{'trade': 0.4037849793852937, 'event': 0.3946855542326453, 'ob': 1.0690682723312026, 'bevent': 0.6803847297050505, 'btrade': -0.0361893235014932, 'spreadreturn': 0.8608047770175122, 'niftyidevent': 0.38319244138034275, 'niftyidindex': 0.29537407020340184}
{'trade': 0.10094624484632342, 'event': 0.06578092570544088, 'ob': 0.2138136544662405, 'bevent': 0.11339745495084175, 'btrade': -0.0060315539169155325, 'spreadreturn': 0.12297211100250174, 'niftyidevent': 0.047899055172542844, 'niftyidindex': 0.04219629574334312}


In [7]:
x=data[feature_cols_JUL]
y= data['return800']
pred=[]
for j in range(len(x)):
    s=0
    for i in x.columns:
        m=davg[i]*x.at[j,i]
        s+=m
    pred.append(s)


In [6]:
l1=[]
l2=[]
for i in range(len(y)):
    if(y[i]>0):
        l1.append(1)
    elif(y[i]<0):
        l1.append(-1)
    else:
        l1.append(0)
        
for i in range(len(pred)):
    if(pred[i]>0):
        l2.append(1)
    elif(pred[i]<0):
        l2.append(-1)
    else:
        l2.append(0)

print(len(l1))
print(len(l2))
pred= pd.Series(data=pred)
pred1 = pd.Series(data=l1)
pred2 = pd.Series(data=l2)
#corr1 = y_model_1.corr(pred1)
#corr2 = y_model_2.corr(pred2)
co= pred1.corr(pred2)
print(co)

457897
457897
0.05505344046540921


##### Predicting return800 values on the basis of new averae weights obtained above

In [8]:
ybar = np.mean(y)
sst=0
ssr=0
for i in y:
    ss= (i-ybar)**2
    sst+=ss
print(sst)
for i in pred:
    ss2= (i-ybar)**2
    ssr+=ss2
print(ssr)
cod= ssr/sst
print(cod)

8033500.013158645
52459.237760916716
0.0065300600827771175


###### Instead of taking the average weights , here the average weights are multiplied by some random constants to observe the changes in predicted values

In [9]:
w=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
print(davg)
k=0
for i,j in davg.items():
    ran=random.randint(0,9)
    davg[i]=(davg[i]*w[k])/sum(w)
    k+=1
print(davg)

{'trade': 0.20405610494078094, 'event': 0.009700596255870829, 'ob': 0.21403620060955314, 'bevent': 0.1299418673828545, 'btrade': -0.06577290791652482, 'spreadreturn': 0.0977985905826335, 'niftyidevent': 0.05838997903392235, 'niftyidindex': 0.03568022172623834}
{'trade': 0.00371011099892329, 'event': 0.00035274895475893923, 'ob': 0.01167470185143017, 'bevent': 0.009450317627843964, 'btrade': -0.00597935526513862, 'spreadreturn': 0.01066893715446911, 'niftyidevent': 0.007431451877044662, 'niftyidindex': 0.005189850432907395}


In [11]:
x=data[feature_cols_JUL]
y= data['return800']
pred=[]
for j in range(len(x)):
    s=0
    for i in x.columns:
        m=davg[i]*x.at[j,i]
        s+=m
    pred.append(s)
pred= pd.Series(data=pred)
corre= y.corr(pred)
print(corre)

0.05147105162570885


In [12]:
ybar = np.mean(y)
sst=0
ssr=0
for i in y:
    ss= (i-ybar)**2
    sst+=ss
print(sst)
for i in pred:
    ss2= (i-ybar)**2
    ssr+=ss2
print(ssr)
cod= ssr/sst
print(cod)

8033500.013158645
232.46308857547223
2.8936713536404345e-05


In [39]:
n=10
p=n
sco=[]
dcount={}
davg={}
ms=[]
intersum=0
y=data.iloc[:,34]
newpoints1= data.index[data['return800'] >= np.percentile(y,75)].tolist()
newpoints2= data.index[data['return800'] <= np.percentile(y,25)].tolist()
newpoints= newpoints1 + newpoints2
print(len(newpoints1))
print(len(newpoints2))
newpoints = list(set(newpoints))
print(len(newpoints))
df= data.iloc[newpoints,:]
Xx = df.iloc[newpoints,:]
Yy = df.iloc[newpoints,:]
reg=LinearRegression()
reg.fit(Xx,Yy)

for i in feature_cols_JUL:
    dcount.update({i : 0})
    davg.update({i : 0})
while(n):
    col=random.randint(1,7)
    row=random.randint(0,228950)
    col_list=[]
    if(n%2==0):
        feature_cols_JUL.reverse()
    for i in range(len(feature_cols_JUL)):
        if(i<=col):
            col_list.append(feature_cols_JUL[i])
    XX=df[col_list]
    X=XX.iloc[0:row,:]
    Y=df.iloc[0:row,34]
    reg=LinearRegression()
    reg.fit(X,Y)
    Coef = pd.Series(reg.coef_,index=col_list)
    #print(Coef)
    #print()
    inc = reg.intercept_
    intersum+=inc
    for index, value in Coef.items():
        dcount[index]+=1
        davg[index]+=value
    from sklearn.metrics import r2_score,mean_squared_error
    y_pred = reg.predict(X)
    mse=mean_squared_error(Y,y_pred)
    rmse= np.sqrt(mse)
    ms.append(rmse)
    print(len(X))
    n=n-1
print(dcount)
print(davg)
for key,val in davg.items():
    temp= dcount[key]
    davg[key]=val/temp
intavg=intersum/p
print(intavg)
print(davg)
print(ms)

114475
114475
228950
133818
41885
94935
96825
73481
26481
134946
147406
203337
32136
{'niftyidindex': 4, 'niftyidevent': 7, 'spreadreturn': 7, 'btrade': 6, 'bevent': 7, 'ob': 7, 'event': 8, 'trade': 7}
{'niftyidindex': 0.13135592868356089, 'niftyidevent': 0.6459016082399629, 'spreadreturn': 0.7866197269971704, 'btrade': -0.7212251484869635, 'bevent': 1.4535771650100024, 'ob': 3.32362978130372, 'event': 0.2659205836202352, 'trade': 2.572961496803913}
0.035150448341404066
{'niftyidindex': 0.03283898217089022, 'niftyidevent': 0.0922716583199947, 'spreadreturn': 0.11237424671388149, 'btrade': -0.12020419141449391, 'bevent': 0.20765388071571464, 'ob': 0.47480425447196, 'event': 0.0332400729525294, 'trade': 0.36756592811484473}
[6.008830769566028, 5.086825697102761, 5.621590927172614, 5.704100815648873, 5.313786003928863, 5.028656091580054, 6.013081645898843, 6.028205977795311, 5.923639251802401, 5.031047985855063]


In [36]:
x=data[feature_cols_JUL]
y= data['return800']
pred=[]
for j in range(len(x)):
    s=0
    for i in x.columns:
        m=davg[i]*x.at[j,i]*(1/ms[0])
        s+=m
    s+=intavg
    pred.append(s)
pred= pd.Series(data=pred)
corre= y.corr(pred)
print("correlation: ",corre)

correlation:  0.056577061845175435


In [37]:
ybar = np.mean(y)
sst=0
ssr=0
for i in y:
    ss= (i-ybar)**2
    sst+=ss
print(sst)
for i in pred:
    ss2= (i-ybar)**2
    ssr+=ss2
print(ssr)
cod= ssr/sst
print("cod: ",cod)

8033500.013158645
4813.473758645643
cod:  0.000599175172809026
